# Example from the paper:

In [1]:
import SymbolicController
import Automaton
import numpy as np

In [2]:
X_bounds = np.array([[0, 10.0], [0, 10.0], [-np.pi, np.pi]])  # state space bounds
U_bounds = np.array([[0.25, 1.0], [-1.0, 1.0]])  # input space bounds
W_bounds = np.array([[-0.05, 0.05], [-0.05, 0.05], [-0.05, 0.05]])  # disturbance bounds

tau = 1.0  # sampling time

cells_per_dim_x = np.array([100, 100, 30])  # number of cells per dimension in state space
cells_per_dim_u = np.array([3, 5]) # number of cells per dimension in input space
angular_dims_x = [2]  # indices of angular dimensions in state space

In [3]:
def f(x, u, w):
    return [x[0]+tau*(u[0]*np.cos(x[2])+w[0]), x[1]+tau*(u[0]*np.sin(x[2])+w[1]), x[2]+tau*(u[1]+w[2])]

def Jx(u):
    return np.array([[1, 0, tau*np.abs(u[0])],
                     [0, 1, tau*np.abs(u[0])],
                     [0, 0, 1]])

def Jw(u):
    return np.array([[tau, 0, 0],
                     [0, tau, 0],
                     [0, 0, tau]])

In [4]:
SpecificationTransition = {
    ('a', 0): 'a', ('a', 1): 'b', ('a', 2): 'c', ('a', 3): 'a', ('a', 4): 'e',
    ('b', 0): 'b', ('b', 1): 'b', ('b', 2): 'e', ('b', 3): 'd', ('b', 4): 'e',
    ('c', 0): 'c', ('c', 1): 'e', ('c', 2): 'c', ('c', 3): 'd', ('c', 4): 'e',
    ('d', 0): 'd', ('d', 1): 'd', ('d', 2): 'd', ('d', 3): 'd', ('d', 4): 'd',
    ('e', 0): 'e', ('e', 1): 'e', ('e', 2): 'e', ('e', 3): 'e', ('e', 4): 'e',
}

SpecificationAutomaton = Automaton.Automaton(SpecificationTransition, 'a', ['d'], 5)

In [ ]:
sets = {
    # label: [lower_bounds], [upper_bounds], not_intersect, is_default
    1: [[4.0, 8.5, -np.pi], [5.0, 9.5, np.pi], True],
    2: [[8.5, 2.0, -np.pi], [9.5, 3.0, np.pi], True],
    3: [[2.0, 0.5, -np.pi], [3.0, 1.5, np.pi], True],
    4: [[3.0, 3.0, -np.pi], [7.0, 7.0, np.pi], False],
    0: None, # default set
}

def relation(x_min, x_max, set):
    # check if the area intersects with the set
    # or is fully within

    for i in range(len(x_min)):
        if set[2]:  # not_intersect
            if x_max[i] < set[0][i] or x_min[i] > set[1][i]:
                return False
        else:  # is within
            if x_min[i] < set[0][i] or x_max[i] > set[1][i]:
                return False
    return True

In [6]:
# Reload all modules to pick up the changes
import importlib
import sys

# Clear all module caches
for module_name in list(sys.modules.keys()):
    if any(mod in module_name for mod in ['AbstractSpace', 'SymbolicController', 'Discretisation', 'Labeling', 'System', 'ProdAutomaton', 'Controller']):
        del sys.modules[module_name]

# Re-import
import AbstractSpace
import SymbolicController
importlib.reload(AbstractSpace)
importlib.reload(SymbolicController)

# Create controller
Controller = SymbolicController.SymbolicController(f, Jx, Jw, X_bounds, U_bounds, W_bounds, cells_per_dim_x, cells_per_dim_u, angular_dims_x, SpecificationAutomaton, relation, sets)
print("✓ Controller initialized successfully!")

Attempting to load existing symbolic model from file...
✓ Loaded existing symbolic model from file.
✓ Loaded existing symbolic model from file.
✓ Controller initialized successfully!
✓ Controller initialized successfully!


In [7]:
# Note: Controller.start() uses cached results if available for instant synthesis
# The first run computes the fixed-point iteration (can take hours for large state spaces)
# Subsequent runs load from ./Models/ directory instantly
import time
import os

start_time = time.time()
Controller.start((0,0,0), 'a', True)
elapsed = time.time() - start_time

print(f"✓ Controller started successfully!")
print(f"Execution time: {elapsed:.3f}s")
print(f"\nController Results:")
print(f"  Value function size: {Controller.V.shape}")
print(f"  Policy size: {Controller.h.shape}")
print(f"  Reachable states: {np.count_nonzero(Controller.V != -1)}")

# Note: First run loads symbolic model (9+ min), then computes synthesis
# Subsequent runs load pre-computed controller from ./Models/ (instant)
if os.path.exists('./Models/V_result.csv'):
    print(f"  Status: Loaded from cache (./Models/)")
else:
    print(f"  Status: Computed from scratch")

Attempting to load existing controller results from file...
✗ No existing controller results found. Computing new controller...


KeyboardInterrupt: 